# Notebook 3 : Modélisation

Ce notebook a pour objectif de développer et comparer différents modèles de segmentation d'images pour identifier les objets présents dans des scènes de rue. Nous utiliserons le dataset Cityscapes, qui contient des images avec des annotations précises pour différents objets (voitures, piétons, bâtiments, etc.).

Nous avons précédemment explorer les données (notebook 1), puis effectuer un prétraitement (notebook 2).


## Objectifs

* Implémenter et comparer différents modèles de segmentation d'images (UNet, SegNet, etc.)
* Tester différentes fonctions de perte :
  - Entropie croisée (categorical cross-entropy)
  - Dice loss
  - Compromis entre les deux loss
* Évaluer les performances des modèles avec différentes métriques
    - Dice coefficient
    - Intersection over Union (IoU)
* Expérimenter avec l'augmentation de données pour améliorer la robustesse des modèles

## Données

Le dataset Cityscapes est utilisé pour ce projet. Il contient des images de scènes de rue avec des annotations pour 8 classes d'objets :
- route (flat)
- humain (human)
- véhicule (vehicle)
- bâtiment (construction)
- objets (object)
- nature (nature)
- ciel (sky)
- vide (void)
  
## Librairies

* TensorFlow et Keras pour la construction et l'entraînement des modèles
* MLflow pour le suivi des expériences
* OpenCV pour le traitement des images
* NumPy pour les opérations numériques
* Matplotlib pour la visualisation des résultats

## Partie 1 : Configuration de l'environnement et des paramètres

### Installation de mflow

In [1]:
!pip install mlflow -q

### Imports et configurations

In [2]:
# Standard library imports
import os
import glob
import math
import random

# Data manipulation and visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Image processing
import cv2

# Machine learning
from sklearn.model_selection import train_test_split

# Deep learning
import tensorflow as tf
from tensorflow import keras

# MLflow
import mlflow
import mlflow.tensorflow
from mlflow.models import infer_signature



# Chemins vers les données (train / test / val)
DATA_DIR = '../data/processed'
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR = os.path.join(DATA_DIR, "val")
TEST_DIR = os.path.join(DATA_DIR, "test")

# Paramètres
## données
SAMPLING = 0.75
NUM_CLASSES = 8
## Tailles des images redimensionnées (identique au notebook 2)
IMG_HEIGHT = 128
IMG_WIDTH = 256
## batch_size / nombre d'epochs
BATCH_SIZE = 4
EPOCHS = 3

# Configuration de MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("segmentation_images_cityscapes")

ModuleNotFoundError: No module named 'seaborn'

## Partie 2 : Chargement et préparation des données

### Chargement et préparation des données

In [ ]:
# Chargement et préparation des données

def load_data(data_dir, sample_ratio = 0.1):
    images = sorted(glob.glob(os.path.join(data_dir, "*", "*_image.png")))
    masks = sorted(glob.glob(os.path.join(data_dir, "*", "*_mask.png")))

    # S'assurer que le nombre d'images et de masques correspond
    assert len(images) == len(masks), "Le nombre d'images et de masques ne correspond pas"

    # Calculer le nombre d'échantillons à sélectionner
    sample_size = int(len(images) * sample_ratio)

    # Créer des paires d'images et de masques
    paired_data = list(zip(images, masks))

    # Mélanger aléatoirement les paires
    random.shuffle(paired_data)

    # Sélectionner un sous-ensemble
    sampled_data = paired_data[:sample_size]

    # Séparer les images et les masques échantillonnés
    sampled_images, sampled_masks = zip(*sampled_data)

    return list(sampled_images), list(sampled_masks)
    return images, masks



## chargement des données preprocéssées
train_images, train_masks = load_data(TRAIN_DIR, sample_ratio = SAMPLING)
val_images, val_masks = load_data(VAL_DIR, sample_ratio = SAMPLING)
test_images, test_masks = load_data(TEST_DIR, sample_ratio = SAMPLING)

def data_generator(images, masks, batch_size, num_classes):
    """
    Transforme un générateur de données (images et masques) en un tf.data.Dataset.
    """

    def generator():
        """Générateur pour le tf.data.Dataset."""
        for i in range(0, len(images)):
            # Chargement de l'image
            image = cv2.imread(images[i])
            image = image / 255.0

            # Chargement du masque en grayscale
            mask = cv2.imread(masks[i], cv2.IMREAD_GRAYSCALE)
            mask = keras.utils.to_categorical(mask, num_classes=num_classes)

            yield image, mask

    # Définir les types de données de sortie
    output_types = (tf.float32, tf.float32)

    # Définir les formes des données de sortie (None pour les dimensions variables)
    # Ici, on suppose que toutes les images ont la même taille
    image_shape = cv2.imread(images[0]).shape
    mask_shape = (image_shape[0], image_shape[1], num_classes)

    output_shapes = (tf.TensorShape(image_shape), tf.TensorShape(mask_shape))

    # Créer le tf.data.Dataset à partir du générateur
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_types=output_types,
        output_shapes=output_shapes
    )

    # Batching
    dataset = dataset.batch(batch_size)

    return dataset



### Résumé d'informations sur le dataset

In [ ]:

def print_dataset_info(images, masks, dataset_name):
    """Affiche les informations d'un dataset"""
    print(f"\n{'='*40}")
    print(f"Informations du dataset {dataset_name}:")
    print(f"{'='*40}")

    # Nombre d'échantillons
    print(f"Nombre d'images: {len(images)}")
    print(f"Nombre de masques: {len(masks)}")

    # Dimensions de la première image
    sample_image = cv2.imread(images[0])
    print(f"\nDimensions image (H, W, C): {sample_image.shape}")

    # Dimensions du premier masque
    sample_mask = cv2.imread(masks[0], cv2.IMREAD_GRAYSCALE)
    print(f"Dimensions masque (H, W): {sample_mask.shape}")

    # Vérification de la cohérence des dimensions
    for img, mask in zip(images[:3], masks[:3]):  # Vérification sur 3 échantillons
        img_shape = cv2.imread(img).shape[:2]
        mask_shape = cv2.imread(mask, cv2.IMREAD_GRAYSCALE).shape
        if img_shape != mask_shape:
            print(f"Attention ! Incohérence de dimensions entre {img} et {mask}")

# Affichage des informations pour chaque dataset
print_dataset_info(train_images, train_masks, "TRAIN")
print_dataset_info(val_images, val_masks, "VALIDATION")
print_dataset_info(test_images, test_masks, "TEST")

# Information sur le nombre de classes
print(f"\n{'='*40}")
print(f"Nombre de classes: {NUM_CLASSES}")
print(f"Taille du batch: {BATCH_SIZE}")
print(f"{'='*40}")

# Vérification supplémentaire des canaux
sample_image = cv2.imread(train_images[0])
sample_mask = cv2.imread(train_masks[0], cv2.IMREAD_GRAYSCALE)
print(f"\nExemple de valeurs de pixel (Image): Min={sample_image.min()} Max={sample_image.max()}")
print(f"Exemple de valeurs de pixel (Masque): Classes uniques={np.unique(sample_mask)}")


## Partie 3 : Définition des modèles

Plusieurs modèles (deep learning) sont utilisés dans notre cas. Ils sont spécifiques à la segmentation sémantique.

### 1- UNet mini

In [ ]:
def build_unet_mini(img_height, img_width, num_classes):
    inputs = keras.layers.Input(shape=(img_height, img_width, 3))

    # Downsampling
    conv1 = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = keras.layers.MaxPooling2D((2, 2))(conv1)

    conv2 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = keras.layers.MaxPooling2D((2, 2))(conv2)

    # Bottleneck
    conv3 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)

    # Upsampling
    up4 = keras.layers.UpSampling2D((2, 2))(conv3)
    merge4 = keras.layers.concatenate([conv2, up4], axis=-1)
    conv4 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(merge4)

    up5 = keras.layers.UpSampling2D((2, 2))(conv4)
    merge5 = keras.layers.concatenate([conv1, up5], axis=-1)
    conv5 = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(merge5)

    # Output
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)

    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model

### 2- Modèle UNet

In [ ]:
def build_unet(img_height, img_width, num_classes):
    """
    Définition du modèle UNET spécifique à la segmentation d'images.
    U-Net : https://fr.wikipedia.org/wiki/U-Net
    """

    ## Entrée
    inputs = keras.layers.Input(shape=(img_height, img_width, 3))

    ## Bloc 1
    conv1 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = keras.layers.MaxPooling2D((2, 2))(conv1)

    ## Bloc 2
    conv2 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = keras.layers.MaxPooling2D((2, 2))(conv2)

    ## Bloc 3
    conv3 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)

    ## Bloc 4
    up4 = keras.layers.UpSampling2D((2, 2))(conv3)
    merge4 = keras.layers.concatenate([conv2, up4], axis=-1)
    conv4 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge4)

    ## Bloc 5
    up5 = keras.layers.UpSampling2D((2, 2))(conv4)
    merge5 = keras.layers.concatenate([conv1, up5], axis=-1)
    conv5 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge5)

    ## Couche de sortie
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)
    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model

### 3- Modèle VGG16/UNet

In [ ]:
def build_vgg16_unet(img_height, img_width, num_classes):
    """
    Construit un modèle VGG16-UNet avec des couches d'upsampling supplémentaires
    pour s'assurer que la taille de la sortie correspond à la taille de l'entrée.
    """
    # Charger le modèle VGG16 pré-entraîné sans la partie classification
    vgg16 = keras.applications.VGG16(input_shape=(img_height, img_width, 3),
                                     include_top=False,
                                     weights='imagenet')

    # Encoder (VGG16)
    vgg16_output = vgg16.output

    # Decoder (UNet-like)
    # Ajouter des couches Conv2DTranspose supplémentaires pour augmenter la taille
    up0 = keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(vgg16_output) # Taille: H/16 * 2, W/16 * 2
    merge0 = keras.layers.concatenate([vgg16.get_layer('block5_conv3').output, up0], axis=-1)
    conv0 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(merge0)

    up1 = keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv0) # Taille: H/8 * 2, W/8 * 2
    merge1 = keras.layers.concatenate([vgg16.get_layer('block4_conv3').output, up1], axis=-1)
    conv1 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(merge1)

    up2 = keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv1) # Taille: H/4 * 2, W/4 * 2
    merge2 = keras.layers.concatenate([vgg16.get_layer('block3_conv3').output, up2], axis=-1)
    conv2 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge2)

    up3 = keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv2) # Taille: H/2 * 2, W/2 * 2
    merge3 = keras.layers.concatenate([vgg16.get_layer('block2_conv2').output, up3], axis=-1)
    conv3 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge3)

    up4 = keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv3) # Taille: H * 2, W * 2
    merge4 = keras.layers.concatenate([vgg16.get_layer('block1_conv2').output, up4], axis=-1)
    conv4 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(merge4)

    # Output
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv4)

    model = keras.models.Model(inputs=vgg16.input, outputs=outputs)
    return model

### 4- MobileNetV2_Unet

In [ ]:
def build_mobilenetv2_unet(img_height, img_width, num_classes):
    """Construit un modèle MobileNetV2-UNet."""
    # Charger le modèle MobileNetV2 pré-entraîné sans la partie classification
    mobilenetv2 = keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                     include_top=False,
                                     weights='imagenet')

    # Récupérer la sortie de MobileNetV2
    mobilenet_output = mobilenetv2.output

    # Reshape de la sortie de MobileNetV2
    reshape = keras.layers.Reshape((img_height // 32, img_width // 32, mobilenet_output.shape[-1]))(mobilenet_output)

    # Decoder (UNet-like) avec 5 couches Conv2DTranspose
    up1 = keras.layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(reshape) # 8x16
    conv1 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(up1)

    up2 = keras.layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(conv1) # 16x32
    conv2 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up2)

    up3 = keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(conv2) # 32x64
    conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up3)

    up4 = keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(conv3) # 64x128
    conv4 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up4)

    up5 = keras.layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(conv4) # 128x256
    conv5 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up5)

    # Output
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)

    model = keras.models.Model(inputs=mobilenetv2.input, outputs=outputs)
    return model


## Partie 4 : Définition des fonctions de perte et des métriques

In [ ]:

## (1) Métriques

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Calcule le coefficient de Dice, une métrique de similarité entre deux ensembles.
    Il est souvent utilisé pour évaluer la performance des modèles de segmentation.

    Args:
        y_true (Tensor): Les valeurs de vérité terrain (ground truth).
        y_pred (Tensor): Les prédictions du modèle.
        smooth (float, optional): Un terme de lissage pour éviter la division par zéro. Defaults to 1e-6.

    Returns:
        float: Le coefficient de Dice, une valeur entre 0 et 1 inclusivement.

    Plage de valeur :
        - 0 : Indique une absence totale de chevauchement entre les prédictions et la vérité terrain.
        - 1 : Indique un chevauchement parfait entre les prédictions et la vérité terrain (les ensembles sont identiques).
    """
    y_true_f = keras.backend.flatten(tf.cast(y_true, tf.float32)) # Conversion de y_true en float32
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) + smooth)

def iou_metric(y_true, y_pred, smooth=1e-6):
    """
    Calcule l'IoU (Intersection over Union), également appelé indice de Jaccard,
    une métrique couramment utilisée pour évaluer la performance des modèles de segmentation.

    Args:
        y_true (Tensor): Les valeurs de vérité terrain (ground truth).
        y_pred (Tensor): Les prédictions du modèle.
        smooth (float, optional): Un terme de lissage pour éviter la division par zéro. Defaults to 1e-6.

    Returns:
        float: L'IoU, une valeur entre 0 et 1 inclusivement.

    Plage de valeur :
        - 0 : Indique une absence totale de chevauchement entre les prédictions et la vérité terrain.
        - 1 : Indique un chevauchement parfait entre les prédictions et la vérité terrain.
    """
    y_true_f = keras.backend.flatten(tf.cast(y_true, tf.float32)) # Conversion de y_true en float32
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    union = keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


## (2) Fonctions de perte (loss)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

### on utilisera également la categorical crossentropy

def mixed_loss(y_true, y_pred):
    """Fonction de perte combinant categorical crossentropy et dice loss."""
    return 0.5 * keras.losses.CategoricalCrossentropy()(y_true, y_pred) + 0.5 * dice_loss(y_true, y_pred)

## Partie 5 : Augmentation des données (Data Augmentation)

On génère des données "augmentées", à savoir en faisant des modifications légères :

- flip horizontal (retournement)

- rotation

- zoom

In [ ]:
## possibilité d'ajouter si besoin de l'augmentation
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1)
])

def augmented_data_generator(images, masks, batch_size, num_classes):
    """
    Crée un générateur de données augmentées en utilisant tf.data.Dataset.
    """

    def augment(image, mask):
        """Fonction pour appliquer l'augmentation de données."""
        augmented_image = data_augmentation(image)
        return augmented_image, mask

    # Créer le dataset à partir du générateur de données de base
    dataset = data_generator(images, masks, batch_size, num_classes)

    # Appliquer l'augmentation de données
    dataset = dataset.map(augment)

    return dataset



## Partie 6 : Entraînement des modèles

### Configuration des modèles

In [ ]:
models = {
    "UNet_mini": build_unet_mini(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES),
    "UNet_base": build_unet(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES),
    "VGG16_UNet": build_vgg16_unet(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES),
    "MobileNetV2_pretrained": build_mobilenetv2_unet(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES)
}


### Configuration des loss (fonctions de perte)

In [ ]:

losses = {
    "categorical_crossentropy": keras.losses.CategoricalCrossentropy(),
    "dice_loss": dice_loss,
    "mixed_loss": mixed_loss
}

### Entraînement des modèles


1. Entraînement **sans** data augmentation

In [ ]:
                                                                                                                                                                                                                                                  # Calculer steps_per_epoch
steps_per_epoch = math.ceil(len(train_images) / BATCH_SIZE)

# Calculer validation_steps
validation_steps = math.ceil(len(val_images) / BATCH_SIZE)

# Créer un exemple d'entrée
input_example = np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, 3).astype(np.float32)


## On itère sur les modèles
for model_name, model in models.items():


    ## On itère (2e boucle) sur les loss
    for loss_name, loss in losses.items():

        registered_name = f"{model_name}_{loss_name}"
        ## on sauve les résultats dans MFlow (model + loss)
        with mlflow.start_run(run_name= registered_name):


            ## génération des données d'entraînement
            train_generator = data_generator(train_images,
                                 train_masks,
                                 BATCH_SIZE,
                                 NUM_CLASSES)

            ## génération des données de validation
            val_generator = data_generator(val_images,
                               val_masks,
                               BATCH_SIZE,
                               NUM_CLASSES)


            ## Complilation
            model.compile(optimizer='adam',
                          loss=loss,
                          metrics=[iou_metric, dice_coefficient])

            ## Fitting


            # Obtenir un exemple de sortie (prédiction)
            output_example = model.predict(input_example)

            # Déduire la signature
            signature = infer_signature(input_example, output_example)

            print(f"fitting {model_name}_{loss_name}")

            #mlflow.keras.autolog()
            model.fit(train_generator,
                      validation_data=val_generator,
                      epochs=EPOCHS,
                     steps_per_epoch=steps_per_epoch,
                     validation_steps = validation_steps)

            # log model

            mlflow.keras.log_model(model,
                                   "model",
                                   registered_model_name=registered_name,
                                   signature=signature,
                                   pip_requirements=["tensorflow", "keras", "opencv-python"])


            # Sauvegarde local du modèle
            print(f"Sauvegarde locale du modèle {registered_name}...")

            path = "/content/drive/My Drive/OC/OC8/models"
            model_save_path = os.path.join(path, f"{registered_name}.keras")
            model.save(model_save_path)

2. Entraînement **avec** data augmentation

In [ ]:

# Créer un exemple d'entrée
input_example = np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, 3).astype(np.float32)


# Entraînement avec augmentation de données
for model_name, model in models.items():


    for loss_name, loss in losses.items():

        ## nom du modèle
        registered_name = f"{model_name}_{loss_name}_augmented"


        with mlflow.start_run(run_name=registered_name):

            ## génération des données d'entraînement augmentées
            augmented_train_generator = augmented_data_generator(train_images,
                                                     train_masks,
                                                     BATCH_SIZE,
                                                     NUM_CLASSES)
            ## génération des données de validation
            val_generator = data_generator(val_images,
                               val_masks,
                               BATCH_SIZE,
                               NUM_CLASSES)


            model.compile(optimizer='adam',
                          loss=loss,
                          metrics=[iou_metric, dice_coefficient])

            ## Fitting


            # Obtenir un exemple de sortie (prédiction)
            output_example = model.predict(input_example)

            # Déduire la signature
            signature = infer_signature(input_example, output_example)

            print(f"fitting {model_name}_{loss_name}_augmented")

            model.fit(augmented_train_generator,
                      validation_data=val_generator,
                      epochs=EPOCHS)

            # log model

            mlflow.keras.log_model(model,
                                   "model",
                                   registered_model_name=f"{model_name}_{loss_name}_augmented",
                                   signature=signature,
                                   pip_requirements=["tensorflow", "keras", "opencv-python"])


            # Sauvegarde local du modèle
            print(f"Sauvegarde locale du modèle {registered_name}...")

            path = "/content/drive/My Drive/OC/OC8/models"
            model_save_path = os.path.join(path, f"{registered_name}.keras")
            model.save(model_save_path)

3. Optimisation d'un hyperparamètre (filter_size pour Unet-mini)

In [ ]:
# Définition des paramètres à tester
filter_sizes = [8, 16, 32]  # Différentes tailles de filtres à tester (paramètre à optimiser)

# Préparation des données (générateurs)
train_generator = data_generator(train_images, train_masks, BATCH_SIZE, NUM_CLASSES)
val_generator = data_generator(val_images, val_masks, BATCH_SIZE, NUM_CLASSES)
steps_per_epoch = math.ceil(len(train_images) / BATCH_SIZE)
validation_steps = math.ceil(len(val_images) / BATCH_SIZE)

# Variables pour suivre le meilleur modèle
best_model = None
best_val_iou = 0.0
best_filter_size = None

# Boucle d'optimisation
for filter_size in filter_sizes:
    with mlflow.start_run(run_name=f"UNetMini_FilterSize_{filter_size}") as run:
        print(f"Training model with filter size: {filter_size}")

        # 1. Construction du modèle (avec le paramètre courant)
        def build_unet_mini_custom(img_height, img_width, num_classes, filter_size):
            inputs = keras.layers.Input(shape=(img_height, img_width, 3))

            ## Bloc 1
            conv1 = keras.layers.Conv2D(filter_size, (3, 3), activation='relu', padding='same')(inputs)
            pool1 = keras.layers.MaxPooling2D((2, 2))(conv1)

            ## Bloc 2
            conv2 = keras.layers.Conv2D(filter_size*2, (3, 3), activation='relu', padding='same')(pool1)
            pool2 = keras.layers.MaxPooling2D((2, 2))(conv2)

            ## Bloc 3
            conv3 = keras.layers.Conv2D(filter_size*4, (3, 3), activation='relu', padding='same')(pool2)

            ## Bloc 4
            up4 = keras.layers.UpSampling2D((2, 2))(conv3)
            merge4 = keras.layers.concatenate([conv2, up4], axis=-1)
            conv4 = keras.layers.Conv2D(filter_size*2, (3, 3), activation='relu', padding='same')(merge4)

            ## Bloc 5
            up5 = keras.layers.UpSampling2D((2, 2))(conv4)
            merge5 = keras.layers.concatenate([conv1, up5], axis=-1)
            conv5 = keras.layers.Conv2D(filter_size, (3, 3), activation='relu', padding='same')(merge5)

            outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)

            model = keras.models.Model(inputs=inputs, outputs=outputs)

            return model

        model = build_unet_mini_custom(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES, filter_size)

        # 2. Compilation du modèle
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[iou_metric])

        # 3. Entraînement du modèle
        history = model.fit(train_generator,
                            validation_data=val_generator,
                            epochs=EPOCHS,
                            steps_per_epoch=steps_per_epoch,
                            validation_steps=validation_steps)

        # 4. Evaluation du modèle (IoU sur la validation)
        val_iou = history.history['val_iou_metric'][-1]
        print(f"Validation IoU: {val_iou}")

        # 5. Log des paramètres et métriques avec MLflow
        mlflow.log_param("filter_size", filter_size)
        mlflow.log_metric("val_iou", val_iou)

        # 6. Sauvegarde du meilleur modèle
        if val_iou > best_val_iou:
            best_val_iou = val_iou
            best_model = model
            best_filter_size = filter_size

            print(f"Nouveau meilleur modèle trouvé avec filter_size={filter_size} et val_iou={best_val_iou}")

# Sauvegarde du meilleur modèle (en local)
if best_model is not None:
    best_model.save("best_unet_mini_model.keras")
    print("Meilleur modèle sauvegardé en local sous le nom : best_unet_mini_model.keras")

    # Log du meilleur modèle avec MLflow
    with mlflow.start_run(run_name="BestUNetMiniModel"): # Démarrer un nouveau run pour le meilleur modèle
        mlflow.log_param("best_filter_size", best_filter_size)
        mlflow.log_metric("best_val_iou", best_val_iou)

        # Créer un exemple d'entrée pour la signature
        input_example = np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, 3).astype(np.float32)
        output_example = best_model.predict(input_example)
        signature = infer_signature(input_example, output_example)

        mlflow.keras.log_model(best_model,
                                "model",
                                registered_model_name="BestUNetMini", # Nom enregistré sur MLflow
                                signature=signature,
                                pip_requirements=["tensorflow", "keras", "opencv-python"])

        print("Meilleur modèle loggé avec MLflow.")
else:
    print("Aucun modèle n'a été entraîné.")

## Partie 7 : Évaluation des modèles

### 1 - Évaluation des modèles sans augmentation

In [ ]:
# Créer une liste pour stocker les résultats
results = []

# Évaluation des modèles entraînés sans augmentation
for model_name, model in models.items():
    for loss_name, loss in losses.items():
        test_generator = data_generator(test_images, test_masks, BATCH_SIZE, NUM_CLASSES)

        search_result = mlflow.search_runs(filter_string=f'tags.mlflow.runName = "{model_name}_{loss_name}"')
        model_path = f"mlruns/839266598857507507/{search_result.iloc[0].run_id}/artifacts/model/data/model.keras"

        loaded_model = keras.models.load_model(model_path,
                                               custom_objects={'iou_metric': iou_metric,
                                                               'dice_loss': dice_loss,
                                                               'mixed_loss': mixed_loss})

        print(f"{model_name}_{loss_name}")
        loss, iou = loaded_model.evaluate(test_generator)
        results.append({
            'Model': model_name,
            'Loss': loss_name,
            'Augmentation': 'No',
            'Loss_Value': loss,
            'IoU': iou
        })

### 2 - Évaluation des modèles avec augmentation

In [ ]:
# Évaluation des modèles entraînés avec augmentation de données
for model_name, model in models.items():
    for loss_name, loss in losses.items():
        test_generator = data_generator(test_images, test_masks, BATCH_SIZE, NUM_CLASSES)

        search_result = mlflow.search_runs(filter_string=f'tags.mlflow.runName = "{model_name}_{loss_name}_augmented"')
        model_path = f"mlruns/839266598857507507/{search_result.iloc[0].run_id}/artifacts/model/data/model.keras"

        loaded_model = keras.models.load_model(model_path,
                                               custom_objects={'iou_metric': iou_metric,
                                                               'dice_loss': dice_loss,
                                                               'mixed_loss': mixed_loss})

        print(f"{model_name}_{loss_name}")
        loss, iou = loaded_model.evaluate(test_generator)
        results.append({
            'Model': model_name,
            'Loss': loss_name,
            'Augmentation': 'Yes',
            'Loss_Value': loss,
            'IoU': iou
        })

### 3- Comparaison visuel des résultats

In [ ]:
# Créer un DataFrame à partir des résultats
df_results = pd.DataFrame(results)

# Créer un barplot pour comparer les IoU
plt.figure(figsize=(12, 6))
sns.barplot(x='Model', y='IoU', hue='Augmentation', data=df_results)
plt.title('Comparaison des IoU par modèle et augmentation')
plt.xlabel('Modèle')
plt.ylabel('IoU')
plt.xticks(rotation=45)
plt.legend(title='Augmentation')
plt.tight_layout()
plt.show()

# Créer un barplot pour comparer les Loss
plt.figure(figsize=(12, 6))
sns.barplot(x='Model', y='Loss_Value', hue='Augmentation', data=df_results)
plt.title('Comparaison des Loss par modèle et augmentation')
plt.xlabel('Modèle')
plt.ylabel('Loss')
plt.xticks(rotation=45)
plt.legend(title='Augmentation')
plt.tight_layout()
plt.show()

### 4- Téléchargement des artifacts

In [ ]:
import os
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_id = "839266598857507507"  # Remplacez par l'ID de votre expérience

# Chemin racine pour sauvegarder tous les artifacts localement
base_dir = "./mlflow_artifacts"

# Parcourir tous les runs de l'expérience et télécharger leurs artifacts
for run in client.search_runs(experiment_ids=[experiment_id]):
    run_id = run.info.run_id
    local_dir = os.path.join(base_dir, run_id)
    mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{run_id}/", dst_path=local_dir)
    print(f"Artifacts du run {run_id} téléchargés dans : {local_dir}")


### 5- Migration du mlflow artifact ==> Google Drive

In [ ]:
import shutil


# Chemin du dossier mlflow_artifacts
source_dir = "/content/mlflow_artifacts"

# Chemin vers Google Drive
destination_dir = "/content/drive/My Drive/OC/OC8/mlflow_artifacts"

# Copier le dossier
shutil.copytree(source_dir, destination_dir)


## Conclusion

Ce notebook a exploré différentes approches pour la segmentation d'images de scènes de rue. Les résultats montrent que l'augmentation de données peut améliorer les performances des modèles.